In [1]:
import os

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.url/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：10


['https://www.dormanproducts.com/p-9421-w37425.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-21537-w72258.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-50659-w610165.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-47833-w610150.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-50424-w610166.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-9423-w37635.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-25698-w19235.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-15216-w37872.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-15277-w37984.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-12090-w79985.aspx?origin=keyword']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = input_.loc[a, 'Url']
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0].strip()
            if type_:
                with open('./part.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        continue

type_

1


'Wheel Cylinder'

In [8]:
dict_src, i, list_src = {}, 0, list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
for src in list_src:
    if src.startswith('http'):
        dict_src[str(i)] = src
        i += 1

dict_src

{'0': 'https://static.dormanproducts.com/images/product/medium/W37425-007.JPG',
 '1': 'https://static.dormanproducts.com/images/product/medium/W37425-006.JPG',
 '2': 'https://static.dormanproducts.com/images/product/medium/W37425-004.JPG'}

In [9]:
part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()

part_number

'W37425'

In [10]:
description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()

description

'Drum Brake Wheel Cylinder'

In [11]:
list_text = [text.strip() for text in html.xpath('//div[@id="divAppSummary"]/p/text()')]
while '' in list_text:
    list_text.remove('')

application_summary = ';'.join(list_text)

application_summary

'BMW 2002 1976-69, BMW 2002ti 1971-69, BMW 2002tii 1974-71'

In [12]:
brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()

brand

'Dorman - First Stop'

In [13]:
vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
vehicle_url = f'''https://www.dormanproducts.com/{vehicle_url[0].strip()}''' if vehicle_url else ''

vehicle_url

'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=9421&PartType=Wheel%20Cylinder'

In [14]:
dict_specification, list_tr, dict_count = {}, html.xpath('//section[@id="productSpec"]/div/table/tr'), {}
for tr in list_tr:
    key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_no = {}
for i, tr in enumerate(list_tr):
    key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
    if dict_count[key] == 1:
        dict_specification[str(i)] = {key: tr.xpath('./td/text()')[0].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_specification[str(i)] = {f'''{key}-{dict_no[key]}''': tr.xpath('./td/text()')[0].strip()}

dict_specification

{'0': {'Bleeder Screw Cap Included': 'No'},
 '1': {'Bleeder Screw Included': 'Yes'},
 '2': {'Brake Component Inlet Thread Size': 'M10 X 1.0'},
 '3': {'Inside Diameter': '0.625 in'},
 '4': {'Material': 'Cast Iron'},
 '5': {'Package Contents': '1 Drum Brake Wheel Cylinder'},
 '6': {'Package Quantity': '1'},
 '7': {'Packaging Type': 'Box'},
 '8': {'Thread Diameter': '10 mm'},
 '9': {'Thread Pitch': '1.0'}}

In [15]:
list_text = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
oem = ';'.join(sorted(list(set(list_text)), key=list_text.index))

oem

'34 21 1 101 760'

In [16]:
import json

df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Type': type_,
                         'Part Number': part_number,
                         'Brand': brand,
                         'Description': description,
                         'Application Summary': application_summary,
                         'OEM': oem,
                         'Picture': '',
                         'Url': request_url,
                         'Json_Src': json.dumps(dict_src),
                         'Vehicle Url': vehicle_url,
                         'Json_Specification': json.dumps(dict_specification)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Type,Part Number,Brand,Description,Application Summary,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification
0,1,Wheel Cylinder,W37425,Dorman - First Stop,Drum Brake Wheel Cylinder,"BMW 2002 1976-69, BMW 2002ti 1971-69, BMW 2002...",34 21 1 101 760,,https://www.dormanproducts.com/p-9421-w37425.a...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Bleeder Screw Cap Included"": ""No""}, ""1..."


In [17]:
crawler_status = 'ok'

crawler_status

'ok'

In [18]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [19]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Type', 'Part Number'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct['No'] = [i+1 for i in range(len(output_correct))]

    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,Type,Part Number,Brand,Description,Application Summary,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification
0,1,Wheel Cylinder,W37425,Dorman - First Stop,Drum Brake Wheel Cylinder,"BMW 2002 1976-69, BMW 2002ti 1971-69, BMW 2002...",34 21 1 101 760,,https://www.dormanproducts.com/p-9421-w37425.a...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Bleeder Screw Cap Included"": ""No""}, ""1..."


In [20]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
